## Project: URL Shortener API

In [ ]:
from flask import Flask, request, jsonify
import sqlite3

app = Flask(__name__)

# Database connection
conn = sqlite3.connect('urls.db')
c = conn.cursor()

# Create table (if it doesn't already exist)
c.execute('''CREATE TABLE IF NOT EXISTS urls (long_url TEXT, short_code TEXT PRIMARY KEY)''')

# Function to generate a unique short code
def generate_short_code():
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    short_code = ''
    while True:
        for _ in range(6):
            short_code += random.choice(alphabet)
        if not c.execute("SELECT * FROM urls WHERE short_code = ?", (short_code,)).fetchone():
            return short_code

# Route to handle URL shortening requests (POST)
@app.route('/shorten', methods=['POST'])
def shorten_url():
    long_url = request.json['long_url']
    if not long_url:
        return jsonify({'error': 'Please provide a long URL'}), 400

    short_code = generate_short_code()
    c.execute("INSERT INTO urls (long_url, short_code) VALUES (?, ?)", (long_url, short_code))
    conn.commit()

    return jsonify({'short_url': f'http://your-api.com/{short_code}'})

# Route to handle shortened URL redirection (GET)
@app.route('/<short_code>')
def redirect_to_long_url(short_code):
    long_url = c.execute("SELECT long_url FROM urls WHERE short_code = ?", (short_code,)).fetchone()
    if not long_url:
        return jsonify({'error': 'Invalid short code'}), 404

    return jsonify({'long_url': long_url[0]})

if __name__ == '__main__':
    app.run(debug=True)


### Functionality:

### Users can submit long URLs through a POST request to the API.
### The API generates a shorter, unique code for the URL and stores it in a database.
### Users can access the shortened URL (e.g., http://your-api.com/abcd1234) to be redirected to the original long ### URL.

### Technology Stack:

### Python (Flask): A versatile and beginner-friendly web framework for building APIs.
### SQLite: A lightweight, embedded database suitable for small-scale projects

#### Explanation:

#### Imports: We import necessary libraries (Flask, request, jsonify from Flask, sqlite3 for database).
#### Flask App Initialization: We create a Flask app instance.
#### Database Connection: We connect to the SQLite database (urls.db) and create a cursor object for interacting with it.
#### Database Table Creation: We define a function to create the urls table if it doesn't exist, specifying columns for long_url (text) and short_code (text as the primary key).
#### Short Code Generation Function: This function generates a random, unique 6-character code using the random module (replace with a more robust generation approach in production).
#### URL Shortening Route (/shorten):
#### This route handles POST requests for shortening URLs.
#### It retrieves the long URL from the request JSON data.
#### If no long URL is provided, it returns an error.
#### It generates a unique short code using the generate_short_code function.
#### It inserts the long URL and short code into the database.
#### It commits the database changes and returns a JSON response with the shortened URL.
#### URL Redirection Route (/<short_code>):
#### This route handles GET requests for shortened URLs (e.g., /abcd1234).
#### It extracts the short code from the URL path.
#### It queries the database using the short code to retrieve the corresponding long URL.
#### If the short code is invalid (not found), it returns an error.
#### Otherwise, it returns a